<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Stock_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=ea1298c00b62f7ccd5ab0a4d130b8928bbd103eddbf02c047dea1eb853bc9af2
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-3*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['AMBUJACEM', 'ASIANPAINT', 'BAJFINANCE', 'BATAINDIA', 'DMART',
       'EICHERMOT', 'GILLETTE', 'GLAND', 'HDFCAMC', 'INDIGOPNTS', 'INFY',
       'IRCTC', 'JUBLFOOD', 'LUXIND', 'PAGEIND', 'PIDILITIND', 'RELIANCE',
       'TATACONSUM', 'VAIBHAVGBL', 'VALIANTORG'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

qualified stocks: 20


,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%
5,3633.0,3356.0,3162.0,3317.0,EICHERMOT.NS,3850.0,3336.21,30.0,100086.30,108990.0,115500.0,8.90,15.40
9,1317.0,1198.0,1104.0,1312.0,INDIGOPNTS.NS,3112.0,1215.89,83.0,100918.87,109311.0,258296.0,8.32,155.94
14,43010.0,40754.0,38755.0,44288.0,PAGEIND.NS,53631.0,39840.50,2.0,79681.00,86020.0,107262.0,7.96,34.61
2,6785.0,6309.0,6020.0,6601.0,BAJFINANCE.NS,7901.0,6365.23,16.0,101843.68,108560.0,126416.0,6.59,24.13
1,3138.0,2968.0,2876.0,3073.0,ASIANPAINT.NS,3566.0,2954.17,33.0,97487.61,103554.0,117678.0,6.22,20.71
15,2544.0,2462.0,2387.0,2540.0,PIDILITIND.NS,2902.0,2442.09,20.0,48841.80,50880.0,58040.0,4.17,18.83
17,786.0,759.0,729.0,765.0,TATACONSUM.NS,875.0,758.64,130.0,98623.20,102180.0,113750.0,3.61,15.34
3,1543.0,1489.0,1442.0,1657.0,BATAINDIA.NS,2173.0,1501.51,67.0,100601.17,103381.0,145591.0,2.76,44.72
12,467.0,456.0,446.0,525.0,JUBLFOOD.NS,903.0,458.16,217.0,99420.72,101339.0,195951.0,1.93,97.09
16,2454.0,2419.0,2350.0,2473.0,RELIANCE.NS,2811.0,2422.52,42.0,101745.84,103068.0,118062.0,1.30,16.04


In [8]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%
19,481.0,499.0,456.0,578.0,VALIANTORG.NS,1851.0,481.71,104.0,50097.84,50024.0,192504.0,-0.15,284.26
7,1355.0,1363.0,1294.0,1722.0,GLAND.NS,4316.0,1354.78,37.0,50126.86,50135.0,159692.0,0.02,218.58
13,1494.0,1401.0,1313.0,1590.0,LUXIND.NS,4546.0,1499.87,34.0,50995.58,50796.0,154564.0,-0.39,203.09
18,320.0,314.0,308.0,321.0,VAIBHAVGBL.NS,966.0,320.64,156.0,50019.84,49920.0,150696.0,-0.20,201.27
9,1317.0,1198.0,1104.0,1312.0,INDIGOPNTS.NS,3112.0,1215.89,83.0,100918.87,109311.0,258296.0,8.32,155.94
12,467.0,456.0,446.0,525.0,JUBLFOOD.NS,903.0,458.16,217.0,99420.72,101339.0,195951.0,1.93,97.09
11,626.0,618.0,605.0,661.0,IRCTC.NS,1164.0,623.94,160.0,99830.40,100160.0,186240.0,0.33,86.56
8,1842.0,1800.0,1766.0,1968.0,HDFCAMC.NS,3229.0,1856.05,27.0,50113.35,49734.0,87183.0,-0.76,73.97
10,1264.0,1251.0,1354.0,1478.0,INFY.NS,1898.0,1252.12,40.0,50084.80,50560.0,75920.0,0.95,51.58
4,3479.0,3550.0,3467.0,3912.0,DMART.NS,5324.0,3600.56,28.0,100815.68,97412.0,149072.0,-3.38,47.87


In [9]:
investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

print('Investment: ', investment)
print('Current: ', current)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

Investment:  1543109.0
Current:  1586528.0
Current Profit/Loss:  43419.0
Current Profit/Loss %:  2.81
Estimate:  2666193.0
Estimate Profit/Loss:  1123084.0
Estimate Profit/Loss %:  72.78


In [10]:
tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Weightage%
2,6785.0,6309.0,6020.0,6601.0,BAJFINANCE.NS,7901.0,6365.23,16.0,101843.68,108560.0,126416.0,6.59,24.13,6.60
16,2454.0,2419.0,2350.0,2473.0,RELIANCE.NS,2811.0,2422.52,42.0,101745.84,103068.0,118062.0,1.30,16.04,6.59
9,1317.0,1198.0,1104.0,1312.0,INDIGOPNTS.NS,3112.0,1215.89,83.0,100918.87,109311.0,258296.0,8.32,155.94,6.54
4,3479.0,3550.0,3467.0,3912.0,DMART.NS,5324.0,3600.56,28.0,100815.68,97412.0,149072.0,-3.38,47.87,6.53
3,1543.0,1489.0,1442.0,1657.0,BATAINDIA.NS,2173.0,1501.51,67.0,100601.17,103381.0,145591.0,2.76,44.72,6.52
5,3633.0,3356.0,3162.0,3317.0,EICHERMOT.NS,3850.0,3336.21,30.0,100086.30,108990.0,115500.0,8.90,15.40,6.49
11,626.0,618.0,605.0,661.0,IRCTC.NS,1164.0,623.94,160.0,99830.40,100160.0,186240.0,0.33,86.56,6.47
12,467.0,456.0,446.0,525.0,JUBLFOOD.NS,903.0,458.16,217.0,99420.72,101339.0,195951.0,1.93,97.09,6.44
17,786.0,759.0,729.0,765.0,TATACONSUM.NS,875.0,758.64,130.0,98623.20,102180.0,113750.0,3.61,15.34,6.39
1,3138.0,2968.0,2876.0,3073.0,ASIANPAINT.NS,3566.0,2954.17,33.0,97487.61,103554.0,117678.0,6.22,20.71,6.32


In [11]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/Stock-Portfolio-Analysis.csv', index=False)